In [ ]:
# need to use selenium as the javascript renders the html after the page load
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from bs4 import BeautifulSoup

# Setup Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode (no window)
options.add_argument("--disable-gpu")  # Optional: disables GPU (for stability)
options.add_argument("--window-size=1920,1080")

# Create WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the page
url = "https://fightodds.io"
driver.get(url)

# Wait for JavaScript to render (adjust time if needed or use WebDriverWait)
time.sleep(5)

# Get the rendered HTML
html = driver.page_source

# Close the browser
driver.quit()

# Now parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


In [100]:
# Example: find the rendered odds section
odds_sections = soup.find_all("thead", class_="MuiTableHead-root")[0]
print(odds_sections.prettify())

<thead class="MuiTableHead-root">
 <tr class="MuiTableRow-root MuiTableRow-head">
  <th class="MuiTableCell-root MuiTableCell-head jss1641 jss1642 undefined jss1644 false MuiTableCell-alignCenter" scope="col">
   Fighters
  </th>
  <th class="MuiTableCell-root MuiTableCell-head jss1641 undefined undefined false false MuiTableCell-alignCenter" scope="col">
   <a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/goto/betonline" target="_blank">
    BetOnline
   </a>
  </th>
  <th class="MuiTableCell-root MuiTableCell-head jss1641 undefined undefined false false MuiTableCell-alignCenter" scope="col">
   <a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/goto/bovada" target="_blank">
    Bovada
   </a>
  </th>
  <th class="MuiTableCell-root MuiTableCell-head jss1641 undefined undefined false false MuiTableCell-alignCenter" scope="col">
   <a class="MuiTypography-root MuiLink-root MuiLink-underli

In [ ]:
print(f"Found {len(odds_sections)} odds containers")
rows = odds_sections.find_all("tr")
bookies = rows[0].find_all("th")
bookies_list = [bookie.get_text() for bookie in bookies[1:]]
print(bookies_list)

Found 1 odds containers
['BetOnline', 'Bovada', 'Bet105', '4Cx', 'Bookmaker', 'BetAnySports', 'BetUS', 'DraftKings', 'FanDuel', 'Pinnacle', 'Betway', 'Circa', 'ESPN', 'BetRivers', 'BetMGM', 'Caesars', '4casters', 'SXBet', 'Cloudbet', '']


In [101]:
# Example: find the rendered odds section
odds_data = soup.find_all("tbody", class_="MuiTableBody-root")[0]
print(odds_data.prettify())

<tbody class="MuiTableBody-root">
 <tr class="MuiTableRow-root jss1629">
  <td class="MuiTableCell-root MuiTableCell-body jss1659 jss1662 undefined jss1660 false jss1664 false">
   <a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/fighters/jamahal-hill-30344">
    Jamahal Hill
   </a>
  </td>
  <td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined jss1660 false false false">
   <button class="MuiButtonBase-root MuiButton-root MuiButton-text" style="padding: 0px; min-width: 55px; font-weight: 400;" tabindex="0" type="button">
    <span class="MuiButton-label">
     <div class="jss1666">
      <div class="jss1667">
       <span class="jss1669 false">
        -136
       </span>
      </div>
      <div>
       <svg aria-hidden="true" class="MuiSvgIcon-root jss1671" focusable="false" viewbox="0 0 24 24">
        <path d="M7 14l5-5 5 5z">
        </path>
       </svg>
      </div>
     </div>
    </span>
   </button>
  <

In [55]:
def get_name(row):
    name = row.find_all("td")[0].find("a").get_text()
    return name

def get_odds(row, bookies_list):
    # td_list = row.find_all("td")
    # odds_list = []
    # for td in td_list[1:]:
    #     odds = td.find_all("span", class_="jss1669 false")[0].get_text()
    #     odds_list.append(odds)
    # return odds_list
    td_list = row.find_all("td")
    td_list
    odds_list = []
    for td in td_list[1:len(bookies_list)+1]:
        odds_results = td.find_all("span")
        if len(odds_results) > 0:
            text = odds_results[0].get_text()
            odds_list.append(text)
        else:
            print("No odds found in this td")
            print(td.prettify())
    return odds_list

def get_fighter_odds_for_card(odds_data):
    print(f"Found {len(odds_data)} odds containers")
    section_rows = odds_sections.find_all("tr")
    # td0 = row0.find_all("td")[0]
    # print(td0.prettify())
    # print(rows[0].prettify())
    bookies = section_rows[0].find_all("th")
    bookies_list = [bookie.get_text() for bookie in bookies[1:-1]] # empty first and last bookies


    rows = odds_data.find_all("tr")
    
    # odds_dict = {}
    data = []

    print(f'bookies_list: {bookies_list}')
    for row in rows:
        name = get_name(row)
        odds_list = get_odds(row, bookies_list) 
        # odds_dict[name] = dict(zip(bookies_list, odds_list))
        df_row = {"Fighter": name}
        for bookie, odds in zip(bookies_list, odds_list):
            df_row[bookie] = odds
        data.append(df_row)
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    # Optional: set Fighter as index
    df.set_index("Fighter", inplace=True)
    return df

odds_sections = soup.find_all("thead", class_="MuiTableHead-root")[0]
odds_data = soup.find_all("tbody", class_="MuiTableBody-root")[0]

df = get_fighter_odds_for_card(odds_data)

Found 24 odds containers
bookies_list: ['BetOnline', 'Bovada', 'Bet105', '4Cx', 'Bookmaker', 'BetAnySports', 'BetUS', 'DraftKings', 'FanDuel', 'Pinnacle', 'Betway', 'Circa', 'ESPN', 'BetRivers', 'BetMGM', 'Caesars', '4casters', 'SXBet', 'Cloudbet']
No odds found in this td
<td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined jss1660 false false false">
</td>

No odds found in this td
<td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined false jss1661 false false">
</td>

No odds found in this td
<td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined jss1660 false false false">
</td>

No odds found in this td
<td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined false jss1661 false false">
</td>



In [56]:
df

,BetOnline,Bovada,Bet105,4Cx,Bookmaker,BetAnySports,BetUS,DraftKings,FanDuel,Pinnacle,Betway,Circa,ESPN,BetRivers,BetMGM,Caesars,4casters,SXBet,Cloudbet
Fighter,,,,,,,,,,,,,,,,,,,
Jamahal Hill,-141,-133,-131,-129,+116,-145,-130,-118,-108,-128,+900,-140,-110,-110,-110,-105,-127,-127,+1354
Khalil Rountree,+121,+115,+116,+118,-146,+115,+110,-102,-108,+114,-5000,+120,-110,-114,-110,-115,+120,+108,-5000
Rafael Fiziev,+118,+122,+130,+125,-115,+108,+118,+120,+110,+121,-5000,+130,+120,+114,+110,+120,+128,+111,+132
Ignacio Bahamondes,-138,-142,-145,-142,-115,-138,-140,-142,-130,-136,+900,-150,-140,-143,-135,-140,-135,-152,-144
Curtis Blaydes,-258,-260,-248,-249,-285,-255,-265,-245,-255,-235,+280,-265,-260,-250,-250,-250,-249,-257,-247
Rizvan Kuniev,+218,+215,+217,+231,+218,+215,+218,+200,+210,+205,-455,+225,+210,+195,+200,+205,+236,+239,+222
Myktybek Orolbai,-180,-165,-182,-183,-165,-175,-175,-162,-164,-157,-400,-175,-160,-165,-160,-160,-179,-188,-179
Tofiq Musayev,+155,+140,+160,+167,+131,+145,+146,+136,+138,+139,+255,+152,+135,+128,+135,+135,+171,+155,+162
Nazim Sadykhov,-440,-440,-391,-413,-425,-470,-440,-425,-430,-401,-450,-420,-450,-435,-425,-420,-404,-400,-386


In [50]:
# odds_sections = soup.find_all("thead", class_="MuiTableHead-root")[0]
# odds_data = soup.find_all("tbody", class_="MuiTableBody-root")[0]    
    
rows = odds_data.find_all("tr")
row = rows[0]
name = row.find_all("td")[0].find("a").get_text()
print(name)

Jamahal Hill


In [51]:
td_list = row.find_all("td")
td_list
# odds_list = []
# for td in td_list[1:]:
#     odds = td.find_all("span", class_="jss1669 false")[0].get_text()
#     odds_list.append(odds)

td2 = row.find_all("td")[1]
td2
odds = td2.find_all("span", class_="jss1669 false")[0].get_text()
print(odds)

-141


In [42]:
td_list = row.find_all("td")
td_list
odds_list = []
for td in td_list[1:len(bookies_list)+1]:
    odds_results = td.find_all("span")
    if len(odds_results) > 0:
        text = odds_results[0].get_text()
        odds_list.append(text)
    else:
        print("No odds found in this td")
        print(td.prettify())
    print(text)

-141
-133
-131
-129
+116
-145
-130
-118
-108
-128
+900
-140
-110
-110
-110
-105
-127
-127
+1354
120


In [ ]:
row2 = odds_data.find_all("tr")[2]
td2 = row2.find_all("td")[0]
print(td2.prettify())

<td class="MuiTableCell-root MuiTableCell-body jss1659 jss1662 undefined jss1660 false jss1664 false">
 <a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/fighters/rafael-fiziev-26481">
  Rafael Fiziev
 </a>
</td>



In [ ]:
row3 = odds_data.find_all("tr")[3]
td3 = row3.find_all("td")[0]
print(td3.prettify())

<td class="MuiTableCell-root MuiTableCell-body jss1659 jss1662 undefined false jss1661 jss1664 false">
 <a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/fighters/ignacio-bahamondes-8984">
  Ignacio Bahamondes
 </a>
</td>



In [ ]:
odds_data

[<tbody class="MuiTableBody-root"><tr class="MuiTableRow-root jss1629"><td class="MuiTableCell-root MuiTableCell-body jss1659 jss1662 undefined jss1660 false jss1664 false"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-colorPrimary" href="/fighters/jamahal-hill-30344">Jamahal Hill</a></td><td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined jss1660 false false false"><button class="MuiButtonBase-root MuiButton-root MuiButton-text" style="padding: 0px; min-width: 55px; font-weight: 400;" tabindex="0" type="button"><span class="MuiButton-label"><div class="jss1666"><div class="jss1667"><span class="jss1669 false">-136</span></div><div><svg aria-hidden="true" class="MuiSvgIcon-root jss1671" focusable="false" viewbox="0 0 24 24"><path d="M7 14l5-5 5 5z"></path></svg></div></div></span></button></td><td class="MuiTableCell-root MuiTableCell-body jss1659 undefined undefined jss1660 false false false"><button class="MuiButtonBase-root Mui

In [27]:
row0 = odds_data.find_all("tr")[0]
odds_idx = 4
td2 = row0.find_all("td")[odds_idx]
odds = td2.find_all("span", class_="jss1669 false")[0].get_text()
print(odds)

-129


In [93]:
bovada_odds_for_fighter_0 = td3.find("span", class_ = "jss1669").get_text()
bovada_odds_for_fighter_0

'-130'